In [1]:
%run util.py

In [32]:
# N = 2
# N = 3
N = 4
# N = 5

In [2]:
# mu_0: the initial distribution; 1 x (N**2) 
# mu: the actual stationary distribution; 1 x (N**2)
# mu_1: the estimated stationary distribution 
# P: the new transition matrix (the old transition matrix is Q)
# G_1: the estimate of the gradient
# H_1: the estimate of the Hessian
# U_1: an estimated sample path of the Gaussian random vector U; length 1000

mu_0, mu, mu_1, P, G_1, H_1, U_1 = ChainGen(N)  # P is the ground truth transition matrix
mu_0P, muP, mu_1P, PP, G_1P, H_1P, U_1P = ChainGen(N)  # PP is a different transition matrix for testing purposes
zdump([N, mu_0, mu, mu_1, P, G_1, H_1, U_1, mu_0P, muP, mu_1P, PP, G_1P, H_1P, U_1P], 'P_PP_%d.pkz'%N)

<string>:352: RuntimeWarning: covariance is not positive-semidefinite.


In [33]:
N, mu_0, mu, mu_1, P, G_1, H_1, U_1, mu_0P, muP, mu_1P, PP, G_1P, H_1P, U_1P = zload('P_PP_%d.pkz'%N)

In [34]:
mu

array([ 0.02967677,  0.05358132,  0.05706779,  0.02110045,  0.0110389 ,
        0.01295778,  0.24216203,  0.03138075,  0.09665995,  0.16025842,
        0.0009534 ,  0.05344314,  0.0240507 ,  0.07074195,  0.01113169,
        0.12379497])

In [35]:
muP

array([ 0.04857175,  0.04955665,  0.01583266,  0.06294882,  0.0756496 ,
        0.04597001,  0.06437472,  0.10750865,  0.04284779,  0.0884667 ,
        0.00121012,  0.06875005,  0.00984074,  0.10950962,  0.11985716,
        0.08910496])

In [36]:
# n_range = range(20, 50, 10)
# n_range = range(30, 90, 20)
n_range = range(40, 130, 30)
# n_range = range(50, 170, 40)

In [6]:
num_test_sample = 2000
beta_list = list(np.arange(0, 0.2, 0.01)[1:-1]) + list(np.arange(0.2, 1.01, 0.05)[:-1])

In [7]:
eta_wc = {}
eta_Sanov = {}
test_sample = {}
for n in n_range:
    # Get sample paths of the Markov chain with length n; 
    # these paths will be the test set
    test_sample[str(n)] = []
    for idx in range(int(num_test_sample/2)):
        test_sample[str(n)].append(chain(mu, P, n))  
    for idx in range(int(num_test_sample/2)):
        test_sample[str(n)].append(chain(muP, PP, n))  
    # Get thresholds for Hoeffding's test corresponding to sample lenth n    
    for beta in beta_list:
        key = str(n) + '_' + str(beta)
        eta_1 = ThresUeakConv(N, beta, n, mu_0, mu, mu_1, P, G_1, H_1, U_1).ThresCal()
        eta_2 = ThresSanov(N, beta, n, mu_0, mu, mu_1, P, G_1, H_1, U_1).ThresCal()
        eta_wc[key] = eta_1
        eta_Sanov[key] = eta_2
zdump([test_sample, eta_wc, eta_Sanov], 'testSample_threshold_%d.pkz'%N)

In [37]:
test_sample, eta_wc, eta_Sanov = zload('testSample_threshold_%d.pkz'%N)

In [9]:
from __future__ import division

mu = np.reshape(mu, (N, N))

TPR_wc = {}
FPR_wc = {}
TPR_Sanov = {}
FPR_Sanov = {}

for n in n_range:
    for beta in beta_list:
        TP_wc = 0
        FP_wc = 0
        TP_Sanov = 0
        FP_Sanov = 0
        key = str(n) + '_' + str(beta)
        for idx in range(num_test_sample):
            KL = KL_est(test_sample[str(n)][idx], mu)
            if idx > 1000 and KL > eta_wc[key]:
                TP_wc += 1
            if idx < 1000 and KL > eta_wc[key]:
                FP_wc += 1
            if idx > 1000 and KL > eta_Sanov[key]:
                TP_Sanov += 1
            if idx < 1000 and KL > eta_Sanov[key]:
                FP_Sanov += 1
        TPR_wc[key] = TP_wc / 1000
        FPR_wc[key] = FP_wc / 1000
        TPR_Sanov[key] = TP_Sanov / 1000
        FPR_Sanov[key] = FP_Sanov / 1000
        
zdump([TPR_wc, FPR_wc, TPR_Sanov, FPR_Sanov], 'ROCpt_%d.pkz'%N)

In [38]:
TPR_wc, FPR_wc, TPR_Sanov, FPR_Sanov = zload('ROCpt_%d.pkz'%N)

In [39]:
TPR_wc_list = {}
FPR_wc_list = {}
TPR_Sanov_list = {}
FPR_Sanov_list = {}
        
for n in n_range:
    TPR_wc_list[str(n)] = []
    FPR_wc_list[str(n)] = []
    TPR_Sanov_list[str(n)] = []
    FPR_Sanov_list[str(n)] = []
    TPR_wc_list[str(n)].append(0)
    FPR_wc_list[str(n)].append(0)
    TPR_Sanov_list[str(n)].append(0)
    FPR_Sanov_list[str(n)].append(0)
    for beta in beta_list:
        key = str(n) + '_' + str(beta)
        TPR_wc_list[str(n)].append(TPR_wc[key])
        FPR_wc_list[str(n)].append(FPR_wc[key])
        TPR_Sanov_list[str(n)].append(TPR_Sanov[key])
        FPR_Sanov_list[str(n)].append(FPR_Sanov[key])

    TPR_wc_list[str(n)].append(1)
    FPR_wc_list[str(n)].append(1)
    TPR_Sanov_list[str(n)].append(1)
    FPR_Sanov_list[str(n)].append(1)

In [40]:
import matplotlib.pyplot as plt
import pylab
from pylab import *

ROC_WC_list = []
ROC_Sanov_list = []
style_list = ['bo-', 'gs-', 'r+-', 'c<-', 'm>-', 'y^-']
idx = 0

pylab.xlim(-0.1, 1.1)
pylab.ylim(-0.1, 1.1)

for n in n_range:
    ROC_WC_list.append(plt.plot(FPR_wc_list[str(n)], TPR_wc_list[str(n)], style_list[idx]))
    ROC_Sanov_list.append(plt.plot(FPR_Sanov_list[str(n)], TPR_Sanov_list[str(n)], style_list[idx+1]))
    idx += 2

legend_list_1 = []
legend_list_2 = []
legend_list_1.append(ROC_WC_list)
legend_list_1.append(ROC_Sanov_list)
legend_list_1 = list(np.array(legend_list_1).ravel())

for n in n_range:
    legend_list_2.append("Use threshold $\eta_n^{wc}$; $n=%d$"%n)
for n in n_range:
    legend_list_2.append("Use threshold $\eta_n^{sv}$; $n=%d$"%n)
plt.legend(legend_list_1, legend_list_2, loc=4, frameon=False)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid()

savefig('ROC_%d.eps'%N)
plt.show()